
# Fehlerbehandlung

#### Patrick Schnider, Marcel Lüthi</br>Departement Mathematik und Informatik, Universität Basel

Bisher haben wir einen Aspekt des Programmierens bewusst ausgelassen.
Unser Interface lässt auch Befehle für eine Liste zu, welche zu Fehlern führen können.
Eine Fehlerbehandlung haben wir noch nicht eingeführt.

Wir schauen uns dies wieder an einer Teilmenge der Operationen unserer Listenklasse an.
Insbesondere interessiert uns die `get` Methode.

In [ ]:
interface List<E> {
    
    /**
      * Appends an element to the end of the list
      */
    void add(E element);
    
    /**
      * returns the number of elements in the list
      */
    int size();
    
    /**
      * gets the element at position i
      */
    E get(int index);
}

Wir können uns fragen: Was passiert, wenn wir einen Index übergeben, der grösser ist als die Anzahl Elemente?

#### Aufgabe: 

* Schauen Sie sich die Implementation `get()` (unten) an. Was macht die Methode im Fehlerfall? Probieren Sie es auch aus.
* Diskutieren Sie Folgendes: 
    * Was sollte die Methode `get()` machen, wenn ein negativer oder zu grosser Index angegeben wird?
    * Manche Methoden können fehlschlagen (wie `get()`) und manche nicht (wie `size()`). Wie kann ein Benutzer dies unterscheiden? 

In [ ]:
class Node<E> {
    E value;
    Node<E> next;
    
    Node(E value) {
        this.value = value;
        this.next = null;
    }
}

In [ ]:
class LinkedList<E> implements List<E> {
    
    Node<E> first;
    Node<E> last;
    
    int size;
    
    // Erzeugt eine ArrayList mit gegebener Kapazität
    public LinkedList() {
        this.first = null;
        this.last = null;
        this.size = 0;
    }
    
    
    // Fügt ein neues Element am Ende der Liste an. 
    public void add(E element) {
        Node<E> newNode = new Node<E>(element);
        if (first == null) {
            first = newNode;
            last = newNode;
        } else {
            last.next = newNode;
            last = newNode;
        }        
        size = size + 1;
    }
    
    public int size() { 
        return size;
    }
    
    public E get(int index) {
        Node<E> curr = first;
        for (int i = 0; i < index; i = i + 1) {
            curr = curr.next;
        }
        return curr.value;
        
    }   
}

In [ ]:
class LinkedListTest {
    public static void main(String[] args) {
        LinkedList<String> ll = new LinkedList();
        ll.add("foobar");
        ll.get(3);
    }
}

LinkedListTest.main(new String[0]);

# Teil 1: Die Klasse Optional

Eine Möglichkeit, dem Benutzer mitzuteilen, dass eine Methode eventuell kein Resultat zurückgeben wird ist, dies durch einen Datentyp, eine Klasse zu repräsentieren. Java bietet dafür die Klasse `Optional` an.

In [ ]:
interface List<E> {
    
    /**
      * Appends an element to the end of the list
      */
    void add(E element);
    
    /**
      * returns the number of elements in the list
      */
    int size();
    
    /**
      * gets the element at position i
      */
    Optional<E> get(int index);
}

Diese Klasse `Optional` kann wie folgt implementiert werden. 

In [ ]:
class Optional<T> {
    T value = null;

    private Optional(T value) {
        this.value = value;
    }
    
    static <T> Optional<T> of(T value) { 
        return new Optional<T>(value);
    }

    static <T> Optional<T> empty() { 
        return new Optional<T>(null);
    }

    boolean isPresent() { 
        return this.value != null;
    }

     public T get() {
        return value;
    }
    
    @Override
    public String toString() { 
        if (value == null) {
            return "empty";
        } else { 
            return value.toString();
        }
    }
}

Nun können wir die Implementation der Listenklasse anpassen. 

In [ ]:
class LinkedList<E> implements List<E> {
    
    Node<E> first;
    Node<E> last;
    
    int size;
    
    // Erzeugt eine ArrayList mit gegebener Kapazität
    public LinkedList() {
        this.first = null;
        this.last = null;
        this.size = 0;
    }
    
    
    // Fügt ein neues Element am Ende der Liste an. 
    public void add(E element) {
        Node<E> newNode = new Node<E>(element);
        if (first == null) {
            first = newNode;
            last = newNode;
        } else {
            last.next = newNode;
            last = newNode;
        }        
        size = size + 1;
    }
    
    public int size() { 
        return size;
    }
        
    public Optional<E> get(int index) {
        if ((index < 0) || (index >= size)) {
            return Optional.empty();
        }
        Node<E> curr = first;
        for (int i = 0; i < index; i = i + 1) {
            curr = curr.next;
        }
        return Optional.of(curr.value);
        
    }   
}

Bei der Verwendung der Liste können wir auf den Fehlerfall testen und reagieren.

In [ ]:
class LinkedListTest {
    public static void main(String[] args) {
        LinkedList<Double> l = new LinkedList<Double>();
        l.add(3.0);
        Optional<Double> maybe = l.get(0);
        
        if (maybe.isPresent()) {
            Double value = maybe.get();
            System.out.println(value);
        } else {
            System.out.println("Value was not in list.");
        }
    }
}

LinkedListTest.main(new String[0]);

# Teil 2: Exceptions

Die Nutzung von Optional ist eine Möglichkeit der Fehlerbehandlung.
Eine weitere Möglichkeit, die in Java oft verwendet wird, ist die Nutzung von Exceptions. 

Wir wollen uns als erstes die Verwendung in Java ansehen.
Dazu importieren wir die ArrayList Klasse von der Java Bibliothek `java.util`. 

In [ ]:
import java.util.ArrayList;

Der folgende Aufruf führt zu einem Fehler:

In [ ]:
class ExceptionTest {
    public static void main(String[] args) {
        ArrayList<String> list = new ArrayList<String>();
        list.add("abc");
        String value = list.get(3); // <-- hier wird die Exception geworfen
        
        System.out.println("after get().");
    }
}

ExceptionTest.main(new String[0]);

Hier wurde eine *Exception* geworfen um den Fehler anzuzeigen. Exceptions sind einfache Objekte, die im Fehlerfall anstatt eines normalen Rückgabewerts einer Methode zurückgegeben werden. Dies werden aber nicht einer Variable zugewiesen, sondern werden von einem speziellen *Exceptionhandler* gefangen. Dies geschieht wie folgt:

In [ ]:
class ExceptionTest {
    public static void main(String[] args) {
        ArrayList<String> list = new ArrayList<String>();
        list.add("abc");
        
        // Fehlerbehandlung
        try {
            String value = list.get(3);
        } catch (IndexOutOfBoundsException e) {  // <-- fangen der Exception
            System.out.println(e.getMessage());
        }
        
        System.out.println("after get().");
    }
}

ExceptionTest.main(new String[0]);
 

Eine Exception ist einfach eine Klasse. 

##### Übung: 

* Schauen Sie sich die [API-Dokumentation](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/Exception.html) an. 
    * Wie erzeugen Sie eine Exception?
    * Welche Methoden gibt es?
    * Welche Klassen erben von Exceptions?
* Versuchen Sie eine Instanz einer Exception zu erstellen. 
* Experimentieren Sie mit den Methoden. Was machen diese?

In [ ]:
class ExceptionExperiments {
    public static void main(String[] args) {
        Exception e = new Exception("foobar");
        System.out.println(e.getMessage());
        System.out.println(e.toString());
        e.printStackTrace();
    }
}

ExceptionExperiments.main(new String[0]);

Der folgende Code illustriert, wie man einen Fehler mit Exceptions anzeigt:

In [ ]:
class ExceptionTest {
    static double  divide(int a, int b) throws Exception {
        if (b == 0) {
            throw new Exception("cannot divide by 0");
        } else { 
            return a / (double) b;
        }
    }
}

Beachten Sie Folgendes: 

* Die `throws`-Klausel in der Methodensignatur. 
* Im Fehlerfall wird mit `throw` ein neues Objekt der Klasse Exception geworfen. 


#### Aufgaben

* Rufen Sie die Methode `divide()` auf und Fangen Sie die Exception (wie oben mit List gezeigt). 
* Können Sie die LinkedList Klasse so anpassen, dass diese einen Fehler in der `get` Methode mittels einer `IndexOutOfBoundsException` anzeigt?

In [ ]:
class ExceptionTest {
    
    static double  divide(int a, int b) throws Exception {
        if (b == 0) {
            throw new Exception("cannot divide by 0");
        } else { 
            return a / (double) b;
        }
    }

    public static void main(String[] args) {
        try {
            System.out.println(divide(4,0));
        } catch (Exception e) {
            System.out.println(e.getMessage());
        }
    }
}
ExceptionTest.main(new String[0]);

In [ ]:
class LinkedList<E> {
    
    Node<E> first;
    Node<E> last;
    
    int size;
    
    // Erzeugt eine ArrayList mit gegebener Kapazität
    public LinkedList() {
        this.first = null;
        this.last = null;
        this.size = 0;
    }
    
    
    // Fügt ein neues Element am Ende der Liste an. 
    public void add(E element) {
        Node<E> newNode = new Node<E>(element);
        if (first == null) {
            first = newNode;
            last = newNode;
        } else {
            last.next = newNode;
            last = newNode;
        }        
        size = size + 1;
    }
    
    public int size() { 
        return size;
    }
        
    public E get(int index) throws IndexOutOfBoundsException {
        if ((index < 0) || (index >= size)) {
            throw new IndexOutOfBoundsException("index " +index + " is not in a list with size " + size);
        }
        
        Node<E> curr = first;
        for (int i = 0; i < index; i = i + 1) {
            curr = curr.next;
        }
        
        return curr.value;
    }   
}

In [ ]:
class LinkedListTest {
    public static void main(String[] args) {
        LinkedList<String> ll = new LinkedList();
        
        ll.add("foo");
        ll.add("bar");
        
        ll.get(2); // löst Exception aus
    }
}

LinkedListTest.main(new String[0]);